In [ ]:
import os
import cv2 as cv
import numpy as np

In [ ]:
people = []
p = []

# Cambiar la ruta a la carpeta donde están todas las carpetas 
# que contienen las fotos para entrenar al modelo
RUTA = r'C:\Users\isaac\Desktop\openCV\train'
haar_cascade = cv.CascadeClassifier('haar_face.xml')
features = []
labels = []


for i in os.listdir(RUTA):
    people.append(i)


def create_train():
    for person in people:
        path = os.path.join(RUTA, person)
        label = people.index(person)
        
        for img in os.listdir(path):
            img_path = os.path.join(path, img)
            
            img_array = cv.imread(img_path)
            gray = cv.cvtColor(img_array, cv.COLOR_BGR2GRAY)
            
            faces_rect = haar_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=3)
            
            for (x, y, w, h) in faces_rect:
                faces_roi = gray[y:y+h, x: x+w]
                features.append(faces_roi)
                labels.append(label)
                
                
create_train()

In [ ]:
features = np.array(features, dtype='object')
labels = np.array(labels)

face_recognizer = cv.face.LBPHFaceRecognizer_create()

face_recognizer.train(features, labels)

face_recognizer.save('face_trained.yml')
np.save('features.npy', features)
np.save('labels.npy', labels)

In [ ]:
# Si se tuviese que leer el archivo generado se haría de la siguiente forma
haar_cascade = cv.CascadeClassifier('haar_face.xml')

features = np.load('features.npy', allow_pickle=True)
labels = np.load('labels.npy', allow_pickle=True)

face_recognizer = cv.face.LBPHFaceRecognizer_create()
face_recognizer.read('face_trained.yml')

In [ ]:
# Cambiar la ruta de la imagen
img = cv.imread(r"C:\Users\isaac\Desktop\openCV\photos/dicaprio.jpg")
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

faces_rect = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

for (x,y,w,h) in faces_rect:
    faces_roi = gray[y:y+h, x: x+w]
    
    label, confidence = face_recognizer.predict(faces_roi)
    print(f"Label = {people[label]} with a confidence of {confidence}")
    
    cv.putText(img, str(people[label]), (30, 30), cv.FONT_HERSHEY_COMPLEX, 1.0, (0, 255, 0), thickness=2)
    cv.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), thickness=2)

cv.imshow('detected face', img)
cv.waitKey(0)